In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import numpy as np

#### 1. Load datasets

In [87]:
# dataset_name = 'iris'
# perc = 100

# df_train = pd.read_csv(f'../datasets/{dataset_name}/regular/{dataset_name}_train_perc{perc}.csv')
# df_test = pd.read_csv('../datasets/' + dataset_name + '/regular/' + dataset_name + '_test.csv')
# df_val = pd.read_csv('../datasets/' + dataset_name + '/regular/' + dataset_name + '_val.csv')

In [119]:
dataset_name = 'pulsar'
perc = 1

df_train = pd.read_csv(f'../datasets/{dataset_name}/extreme/{perc}/{dataset_name}_train_perc{perc}.csv')
df_test = pd.read_csv(f'../datasets/{dataset_name}/extreme/{perc}/{dataset_name}_test_perc{perc}.csv')

In [120]:
n_classes = len(set(df_train[df_train.columns[-1]]))
print(n_classes)
len(set(df_test[df_test.columns[-1]]))

2


2

In [121]:
if any(isinstance(df_train[x][0], str) for x in df_train.columns):
    ### ----- IF DATA IS CATEGORICAL ----- ##
    print("Categorical data - let's one hot encode the features!\n")
    print("Number of columns before:", len(df_train.columns))
    df_train['set type'] = 'train'
    df_test['set type'] = 'test'
#     df_val['set type'] = 'val'

#     total_df = pd.concat([df_train, df_test, df_val])
    total_df = pd.concat([df_train, df_test])
    cols = list(total_df.columns)
    total_df = pd.get_dummies(total_df, columns=total_df.columns[:-2])
    
    cols = list(total_df.columns)
    cols = cols[1:] + cols[:1]
    total_df = total_df[cols]
    
    df_train = total_df[total_df['set type'] == 'train']
    df_test = total_df[total_df['set type'] == 'test']
#     df_val = total_df[total_df['set type'] == 'val']

    del df_train['set type']
    del df_test['set type']
#     del df_val['set type']
    
    print("Number of columns after one hot encoding:", len(df_train.columns))

In [122]:
target_name = df_train.columns[-1]
X_train, y_train = df_train.drop([target_name], axis='columns'), df_train[target_name]
X_test, y_test = df_test.drop([target_name], axis='columns'), df_test[target_name]
# X_val, y_val = df_val.drop([target_name], axis='columns'), df_val[target_name]

In [123]:
classes_converter = {}
for k, c in enumerate(set(df_train[target_name])):
    classes_converter[c] = k

In [124]:
y_train = y_train.apply(lambda x: classes_converter[x])
y_test = y_test.apply(lambda x: classes_converter[x])

#### 2. Train data using the randomforest classifier

In [125]:
# Neural network module
from keras.utils import np_utils
from keras.models import Sequential 
from keras.optimizers import Adam
from keras.layers import Dense,Activation,Dropout 
from keras.layers.normalization import BatchNormalization 

In [126]:
# Change the label to one hot vector
y_train = np_utils.to_categorical(y_train, num_classes=n_classes)
y_test = np_utils.to_categorical(y_test, num_classes=n_classes)


print("Shape of y_train", y_train.shape)
print("Shape of y_test", y_test.shape)

Shape of y_train (92, 2)
Shape of y_test (9181, 2)


In [127]:
n_cols = X_train[0:1].shape[1]

In [131]:
model=Sequential()
model.add(Dense(1000, input_dim=n_cols, activation='relu'))
# model.add(Dense(100, input_dim=n_cols, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(n_classes,activation='softmax'))
opt = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 1000)              9000      
_________________________________________________________________
dense_42 (Dense)             (None, 2)                 2002      
Total params: 11,002
Trainable params: 11,002
Non-trainable params: 0
_________________________________________________________________


In [132]:
model.fit(X_train, y_train, validation_data = (X_test,y_test), 
          batch_size=32, 
          epochs=50, 
          verbose=1)

Epoch 1/50
3/3 [==============================] - 1s 244ms/step - loss: 3.2610 - accuracy: 0.6630 - val_loss: 0.7473 - val_accuracy: 0.9083
Epoch 2/50
3/3 [==============================] - 1s 203ms/step - loss: 0.6800 - accuracy: 0.9130 - val_loss: 0.7026 - val_accuracy: 0.9171
Epoch 3/50
3/3 [==============================] - 1s 192ms/step - loss: 0.4922 - accuracy: 0.9457 - val_loss: 0.5091 - val_accuracy: 0.9476
Epoch 4/50
3/3 [==============================] - 1s 197ms/step - loss: 0.3221 - accuracy: 0.9565 - val_loss: 0.5387 - val_accuracy: 0.9378
Epoch 5/50
3/3 [==============================] - 1s 205ms/step - loss: 0.2425 - accuracy: 0.9674 - val_loss: 0.7294 - val_accuracy: 0.9288
Epoch 6/50
3/3 [==============================] - 1s 219ms/step - loss: 0.3120 - accuracy: 0.9674 - val_loss: 0.8441 - val_accuracy: 0.9253
Epoch 7/50
3/3 [==============================] - 1s 223ms/step - loss: 0.3050 - accuracy: 0.9565 - val_loss: 0.8508 - val_accuracy: 0.9243
Epoch 8/50
3/3 [====

In [133]:
y_true = np.argmax(y_test, axis=1)
prediction = model.predict(X_test)
y_pred = np.argmax(prediction, axis=1)

accuracy = np.sum(y_true == y_pred)/len(prediction) * 100 
print("Accuracy of the dataset", accuracy)

Accuracy of the dataset 95.38176669208147


On validation set

In [86]:
# model.score(X_val, y_val)

#### 3. Evaluate test set with confusion matrix

In [16]:
y_true.shape

(3754,)

In [17]:
# y_pred = model.predict(X_test)

In [18]:
# cm = confusion_matrix(y_test, y_pred)
# cm

#### 4. Plot results

In [19]:
sn.heatmap(cm, annot=True)
plt.xlabel('pred')
plt.ylabel('true')

NameError: name 'cm' is not defined